In [22]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import requests
import csv


In [26]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

tsoup = BeautifulSoup(source,'lxml')

tablesoup = tsoup.find('table', class_ = 'wikitable sortable')

toronto_csv = open('toronto_neighborhood.csv', 'w')
csv_writer = csv.writer(toronto_csv)

for tr in tablesoup.find_all('tr'):

    Postcode = tr.find_all(['td','th'])[0].text
    Borough = tr.find_all(['td','th'])[1].text
    Neighbourhood = tr.find_all(['td','th'])[2].text
    if Borough != 'Not assigned':
       
    
    csv_writer.writerow([Postcode,Borough,Neighbourhood])
        
toronto_csv.close()

IndentationError: expected an indented block (<ipython-input-26-41ca79972f34>, line 17)

In [16]:
df_t=pd.read_csv('toronto_neighborhood.csv')

In [17]:
df_t.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [18]:
df_t = df_t.replace('\n','', regex=True)
df_t.columns=['Postalcode','Borough','Neighborhood']
df_t.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [19]:
for index, row in df_t.iterrows():
    if df_t.loc[index,'Neighborhood'] == 'Not assigned':
        df_t.loc[index,'Neighborhood'] = df_t.loc[index,'Borough']

# Check to make sure there is no 'Not assigned' in Neighborhood column
df_t.loc[df_t['Neighborhood'] == 'Not assigned']

,Postalcode,Borough,Neighborhood


In [21]:
grouped = df_t.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join)

groupedDF = grouped.to_frame().reset_index()

groupedDF.head(20)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
